In [1]:
!pip install pandas
!!pip install accelerate transformers librosa


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


['Requirement already satisfied: accelerate in /usr/local/lib/python3.10/dist-packages (1.2.1)',
 'Requirement already satisfied: transformers in /usr/local/lib/python3.10/dist-packages (4.48.0)',
 'Requirement already satisfied: librosa in /usr/local/lib/python3.10/dist-packages (0.10.2.post1)',
 'Requirement already satisfied: numpy<3.0.0,>=1.17 in /usr/local/lib/python3.10/dist-packages (from accelerate) (1.24.1)',
 'Requirement already satisfied: packaging>=20.0 in /usr/local/lib/python3.10/dist-packages (from accelerate) (23.2)',
 'Requirement already satisfied: psutil in /usr/local/lib/python3.10/dist-packages (from accelerate) (5.9.6)',
 'Requirement already satisfied: pyyaml in /usr/local/lib/python3.10/dist-packages (from accelerate) (6.0.1)',
 'Requirement already satisfied: torch>=1.10.0 in /usr/local/lib/python3.10/dist-packages (from accelerate) (2.1.0+cu118)',
 'Requirement already satisfied: huggingface-hub>=0.21.0 in /usr/local/lib/python3.10/dist-packages (from acceler

In [2]:
import pandas as pd

In [3]:
recordings_data = {
    "audio_id": [], 
    "chunk_id": [], 
    "transcript": [] 
}

In [4]:
import os 
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
# from datasets import load_dataset
import librosa
import numpy as np
from pydub import AudioSegment
from io import BytesIO

# Set up device and data types
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# Load model and processor
model_id = "openai/whisper-large-v3"  # Use your custom Whisper model
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

# Create ASR pipeline
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=0 if torch.cuda.is_available() else -1,
)

for root, _, files in os.walk("./data/audio_chunks"):
    for i, file in enumerate(files):
        file_path = os.path.join(root, file)
        print(file_path)
        audio = open(file_path, "rb")
        audio = BytesIO(audio.read())
        audio_segment = AudioSegment.from_file(audio)

        # Convert Pydub AudioSegment to NumPy array
        samples = np.array(audio_segment.get_array_of_samples(), dtype=np.float32)
        samples /= np.iinfo(audio_segment.array_type).max  # Normalize to [-1, 1]
    
        # Resample to 16 kHz
        waveform = librosa.resample(samples, orig_sr=audio_segment.frame_rate, target_sr=16000)
        generate_kwargs = {
            # "max_new_tokens": 445,
            "language": "hindi", 
            "task": "translate",
            "num_beams": 1,
            "condition_on_prev_tokens": True,
            "compression_ratio_threshold": 1.35,  # zlib compression ratio threshold (in token space)
            "temperature": (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
            "logprob_threshold": -1.0,
            "no_speech_threshold": 0.6,
            "return_timestamps": True,
        }
        
        result = pipe(waveform, generate_kwargs=generate_kwargs)
        # Run transcription pipeline
        # result = pipe({"array": waveform, "sampling_rate": 16000})

        audio_chunk_id = file_path.split("/")[-1][:-4]
        audio_id = ''.join(audio_chunk_id.split("-")[:-1])
        chunk_id = audio_chunk_id.split("-")[-1]
        print(audio_id, "-", chunk_id)
        
        recordings_data['audio_id'].append(audio_id)
        recordings_data['chunk_id'].append(chunk_id)
        recordings_data['transcript'].append(result["text"])

        recordings_df = pd.DataFrame(recordings_data)

        recordings_df.to_csv("./recordings_data.csv")
        
        print(i)    

Device set to use cuda:0


./data/audio_chunks/92005f1a-713e-47cb-8366-7a4300eedc84-1.wav


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=translate, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=translate.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


92005f1a713e47cb83667a4300eedc84 - 1
0
./data/audio_chunks/181019ba-1580-4926-87bf-ab6a9d8d6451-3.wav


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


...

3892281a35f5427caad95dac2ae6e198 - 3
1359
./data/audio_chunks/3892281a-35f5-427c-aad9-5dac2ae6e198-2.wav


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


3892281a35f5427caad95dac2ae6e198 - 2
1360
./data/audio_chunks/3892281a-35f5-427c-aad9-5dac2ae6e198-1.wav


/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


3892281a35f5427caad95dac2ae6e198 - 1
1361
